In [1]:
from neuroml import Morphology, Segment, Point3DWithDiam as P
from pyNN.morphology import NeuroMLMorphology, uniform
#from pyNN.units import uF_per_cm2, ohm_cm, S_per_cm2, mV, nA, ms
from pyNN.utility import get_simulator
from pyNN.utility.plotting import Figure, Panel

In [2]:
from pyNN import neuron as sim

In [3]:
soma = Segment(proximal=P(x=0, y=0, z=0, diameter=18.8),
               distal=P(x=18.8, y=0, z=0, diameter=18.8),
               name="soma")
dend = Segment(proximal=P(x=0, y=0, z=0, diameter=2),
               distal=P(x=-500, y=0, z=0, diameter=2),
               name="dendrite",
               parent=soma)

In [4]:
cell_class = sim.MultiCompartmentNeuron
cell_class.label = "ExampleMultiCompartmentNeuron"
cell_class.ion_channels = {'pas': sim.PassiveLeak, 'na': sim.NaChannel, 'kdr': sim.KdrChannel}

In [5]:
cell_type = cell_class(morphology=NeuroMLMorphology(Morphology(segments=(soma, dend))),  # yuck
                       cm=1.0,
                       Ra=500.0,
                       pas={"conductance_density": uniform('all', 0.0003),
                            "e_rev":-54.3},
                       na={"conductance_density": uniform('soma', 0.120),
                           "e_rev": 50.0},
                       kdr={"conductance_density": uniform('soma', 0.036),
                            "e_rev": -77.0}
                       )

In [6]:
cells = sim.Population(2, cell_type, initial_values={'v': [-60.0, -70.0]})  #*mV})

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [8]:
step_current = sim.DCSource(amplitude=0.1, start=50.0, stop=150.0)
step_current.inject_into(cells[0:1], location="soma")
step_current.inject_into(cells[1:2], location="dendrite")

AttributeError: 'MultiCompartmentNeuron' object has no attribute 'ionic_species'

In [12]:
cells.record('spikes')
cells.record(['na.m', 'na.h', 'kdr.n'], locations=['soma'])
cells.record('v', locations=['soma', 'dendrite'])

{'pas': PassiveLeak(<parameters>),
 'na': NaChannel(<parameters>),
 'kdr': KdrChannel(<parameters>)}

In [13]:
sim.run(200.0)

<ParameterSpace morphology, cm, Ra, shape=None>

In [ ]:
data = cells.get_data().segments[0]  # this is a Neo Segment

In [ ]:
Figure(
        Panel(data.filter(name='soma.v')[0],
              ylabel="Membrane potential, soma (mV)",
              yticks=True, ylim=(-80, 40)),
        Panel(data.filter(name='dendrite.v')[0],
              ylabel="Membrane potential, dendrite (mV)",
              yticks=True, ylim=(-70, -45)),
        Panel(data.filter(name='soma.na.m')[0],
              ylabel="m, soma",
              yticks=True, ylim=(0, 1)),
         Panel(data.filter(name='soma.na.h')[0],
               xticks=True, xlabel="Time (ms)",
               ylabel="h, soma",
               yticks=True, ylim=(0, 1)),
        Panel(data.filter(name='soma.kdr.n')[0],
              ylabel="n, soma",
              xticks=True, xlabel="Time (ms)",
              yticks=True, ylim=(0, 1)),
        title="Responses of two-compartment neurons to current injection",
        #annotations="Simulated with %s" % options.simulator.upper()
    )

In [ ]:
sim.end()

In [7]:
import inspect

In [8]:
inspect.getfile(sim.CondExpPostSynapticResponse)

'/envs/py3-pyNNarbor/lib/python3.8/site-packages/pyNN/neuron/standardmodels/ion_channels.py'

In [ ]:
        # insert post-synaptic mechanisms
        for name, pse in self.post_synaptic_entities.items():
            parameters = other_parameters[name]
            mechanism_name = pse.model
            synapse_model = getattr(h, mechanism_name)
            density_function = parameters["density"]
            for index, id in enumerate(self.sections):
                density = density_function.value_in(self.morphology, index)
                if density > 0:
                    n_synapses, remainder = divmod(density, 1)
                    rnd = numpy.random  # todo: use the RNG from the parent Population
                    if rnd.uniform() < remainder:
                        n_synapses += 1
                    section = self.sections[id]
                    for i in range(int(n_synapses)):
                        self.morphology.synaptic_receptors[name][id].append(synapse_model(0.5, sec=section))

In [9]:
post_synaptic_entities = {'AMPA': sim.CondExpPostSynapticResponse,
                          'GABA_A': sim.CondExpPostSynapticResponse}

In [ ]:
for name, pse in post_synaptic_entities.items():
    parameters = 

In [10]:
from pyNN.morphology import uniform

In [11]:
AMPA={"density": uniform('all', 0.05),  # number per µm
      "e_rev": 0.0, "tau_syn": 2.0}

In [12]:
AMPA["density"]

In [14]:
AMPA["density"].value

0.05

In [16]:
AMPA["density"].selector

'all'

In [17]:
from pyNN.morphology import load_morphology, uniform, random_section, dendrites, apical_dendrites, by_distance

In [19]:
GABA_A={"density": by_distance(dendrites(), lambda d: 0.05 * (d < 50.0)),  # number per µm
        "e_rev": -70.0, "tau_syn": 5.0}

In [20]:
GABA_A["density"].selector

In [21]:
help(GABA_A["density"].selector)

Help on dendrites in module pyNN.morphology object:

class dendrites(MorphologyFilter)
 |  dendrites(fraction_along=None)
 |  
 |  Method resolution order:
 |      dendrites
 |      MorphologyFilter
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, morphology, filter_by_receptor_type=False)
 |      Return an index (integer NumPy array) that can be used
 |      to retrieve the sections corresponding to the filter.
 |  
 |  __init__(self, fraction_along=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from MorphologyFilter:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

